# Channels

> Channel data structure.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp channels

In [ ]:
#| export

import abc
import asyncio
import enum
import json
import time
import uuid
from dataclasses import dataclass, field
from typing import Any, Generic, TypeVar, Sequence

import fastagent_hacking.streams as sx

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from fastcore.test import *

## Packets

In [ ]:
#| export

_T = TypeVar('T')


class PacketType(enum.StrEnum):
  DATA = enum.auto()
  LOG_PACKET = enum.auto()
  EVENT_PACKET = enum.auto()
  CANCELLATION_PACKET = enum.auto()


def _current_time_ms() -> float:
  return time.time() * 1000


@dataclass(frozen=True)
class Packet(Generic[_T]):
  """Represents a unit of data inside a Channel
  
  Attributes:
    payload: The data that the packet carries.
    packet_type: The type of the packet.
    packet_id: ID for the packet. Default is a random UUID.
    parent_packet_id: ID of the packet that this packet is derived from if any.
      Default is None.
    created_at: The time the packet was created in milliseconds.
    priority: A number that indicates the priority of the packet relative to other 
      _buffered_ packets in the same channel. The higher the number, the higher the priority.
    tags: A sequence of indenpendent labels that can be used to filter packets with.
      Derived packets do not necessarily inherit the tags of their parent packets.
      It's up to the packet creator to decide whether to copy the tags or not.
  """
  payload: _T  # FIXME: This type should be serializeable.
  packet_type: PacketType
  packet_id: str = field(default_factory=lambda: str(uuid.uuid4()))
  parent_packet_id: str | None = None
  created_at: float = field(default_factory=_current_time_ms)
  priority: int = 0
  tags: Sequence[str] = ()

  def __post_init__(self):
    if not isinstance(self.tags, tuple):
      # Ensure immutability.
      object.__setattr__(self, 'tags', tuple(self.tags))

  def to_json(self):
    return json.dumps(
        self,
        default=lambda o: o.__dict__,
        sort_keys=True,
        indent=2,
    )

  @classmethod
  def from_json(cls, json_str):
    return cls(**json.loads(json_str))

  def __lt__(self, other: "Packet[Any]") -> bool:
    """Compare packets for priority queue ordering.
      Higher priority comes first, then older packets."""
    if self.priority != other.priority:
      return self.priority > other.priority
    return self.created_at < other.created_at

In [ ]:
p = Packet(
    payload=1,
    packet_type=PacketType.DATA,
    packet_id='1',
    parent_packet_id='1',
)

test_eq(p, Packet.from_json(p.to_json()))

In [ ]:
def fake_packet(payload, *, priority=0, tags=()) -> Packet:
  return Packet(
      payload=payload,
      packet_type=PacketType.DATA,
      priority=priority,
      tags=tags,
  )

In [ ]:
# Same priority, older packet comes first.
p0 = fake_packet(0)
p1 = fake_packet(1)

test_eq(p0 < p1, True)

In [ ]:
# Higher priority comes first.
p0 = fake_packet(0, priority=0)
p1 = fake_packet(1, priority=1)

test_eq(p1 < p0, True)

In [ ]:
p0 = fake_packet(0, priority=1)
p1 = fake_packet(1, priority=0)
p2 = fake_packet(2, priority=0)
p3 = fake_packet(3, priority=3)

test_eq(sorted([p0, p1, p2, p3]), [p3, p0, p1, p2])

### STD Packets

#### Cancellation Packets

In [ ]:
#| export


def mk_cancellation_packet(*, tag: str) -> Packet:
  """Creates a packet that can be used to cancel packets with the same tag.

  Cancellation packets are propagated through the network through the side channels.
  Channels that receive a cancellation packet should cancel all packets with the same `tag`.
  """
  return Packet(
      payload=tag,
      packet_type=PacketType.CANCELLATION_PACKET,
      packet_id=str(uuid.uuid4()),
      priority=128,
  )

## Channels


In [ ]:
#| export


class Channel(sx.Stream[Packet[Any]], Generic[_T]):
  pass


def as_chan(s: sx.Stream[Packet[Any]]) -> Channel[_T]:
  """Coerce a stream of packets to a channel. Do not use `s` after this function."""

  class _ChanStream(Channel[_T]):

    def __init__(self):
      super().__init__()
      self._pq = asyncio.PriorityQueue()
      self._bad_tags = set()  # FIXME: This can grow indefinitely.

      asyncio.create_task(
          self._pull_from_stream(s)).add_done_callback(_print_task_errors)

    async def next(self, with_status: bool = False) -> Packet[Any]:
      packet, status = None, None
      try:
        packet = await self._pq.get()
        status = sx.StreamStatus.OK
      except asyncio.QueueShutDown:
        packet, status = None, sx.StreamStatus.SHUTDOWN

      if (p := packet):
        if p.packet_type == PacketType.CANCELLATION_PACKET:
          self._bad_tags.add(p.payload)
        elif self._bad_tags & set(p.tags):
          # Skip this packet and try the next one.
          return await self.next(with_status=with_status)

      if with_status:
        return packet, status
      return packet

    async def _pull_from_stream(self, s: sx.Stream[Packet[Any]]):
      async for p in s:
        await self._pq.put(p)
      self._pq.shutdown()

  return _ChanStream()


# FIXME: This is a hack to print errors in async tasks.
def _print_task_errors(task: asyncio.Task):
  if task.exception():
    task.print_stack()
    print(f"Task failed with exception: {task.exception()}")


In [ ]:
#| export


class ChannelWriter(sx.StreamWriter[Packet[Any]], Generic[_T]):
  elm_type: type[_T]  # Main packet payload type of the channel

  @abc.abstractmethod
  def readonly(self) -> Channel[_T]:
    """Return a readonly version of the channel"""


def as_chan_writer(s: sx.StreamWriter[Packet[Any]]) -> ChannelWriter[_T]:
  """Coerce a stream writer of packets to a channel writer. Do not use `s` after this function."""

  class _ChanWriter(ChannelWriter[_T]):

    async def put(self, *args, **kwargs):
      assert all(isinstance(a, Packet) for a in args)
      await s.put(*args, **kwargs)

    async def shutdown(self, *args, **kwargs):
      await s.shutdown(*args, **kwargs)

    def readonly(self, *args, **kwargs) -> Channel[_T]:
      return as_chan(s.readonly(*args, **kwargs))

  return _ChanWriter()

### Channel tests

In [ ]:
p0 = fake_packet(0)
p1 = fake_packet("a")

chan = as_chan(sx.of(p0, p1))

test_eq(isinstance(chan, Channel), True)

ps = []
async for p in chan:
  ps.append(p)
test_eq(ps, [p0, p1])

In [ ]:
chan_writer = as_chan_writer(sx.InMemStreamWriter())

test_eq(isinstance(chan_writer, ChannelWriter), True)

p0 = fake_packet("a")
p1 =  fake_packet("b")

await chan_writer.put(p0, p1)
await chan_writer.shutdown()

ps = []
chan = chan_writer.readonly()
async for p in chan:
  ps.append(p)
test_eq(ps, [p0, p1])

In [ ]:
p0 = fake_packet(0)
p1 = fake_packet(1)
p2 = fake_packet(2)
p3 = fake_packet(3)

c0 = as_chan(sx.of(p0, p1))
c1 = as_chan(sx.of(p2, p3))

ps = []
async for payload in sx.concat(c0, c1):
  ps.append(payload)

test_eq(ps, [p0, p1, p2, p3])

In [ ]:
p0 = mk_cancellation_packet(tag="xyz")
p1 = fake_packet(1, tags=("xyz",))
p2 = fake_packet(2, tags=("xyz",))
p3 = fake_packet(3, tags=("abc",))
p4 = fake_packet(4)

chan = as_chan(sx.of(p0, p1, p2, p3, p4))

ps = []
async for p in chan:
  ps.append(p)

test_eq(ps, [p0, p3, p4])

In [ ]:
p0 = fake_packet(0, tags=("xyz",))
p1 = mk_cancellation_packet(tag="xyz") 
# Cancellation packets have higher priority than normal packets.

chan = as_chan(sx.of(p0, p1))

await asyncio.sleep(0.01) # Wait for the cancellation packet to be propagated.

ps = []
async for p in chan:
  ps.append(p)

test_eq(ps, [p1])

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()